# Un'analisi IRT dei quesiti del Bebras italiano

L'*Item Response Theory* (IRT) è una tecnica statistica che si propone di stimare l'*abilità* dei soggetti cui viene sottoposto un test composto da una serie di domande (dette *item*). L'abilità è un tratto *latente*, cioè non osservabile direttamente: l'aver risposto correttamente a molte domande può dipendere sia da elevata abilità, che dalla "facilità" delle domande. La stima si basa quindi su un *modello matematico* (detto *Item Response Function*) della relazione fra abilità e probabilità di rispondere correttamente alla domanda.

Il modello usato in questa analisi è una curva *logistica*, la cui forma è regolata da tre parametri $a, b, c$.
$$ p_i(\theta_j) = c_i + \frac{(1 - c_i)}{1+ e^{-b_i\cdot(\theta_j - a_i)}} $$
La probabilità $p$ di successo in un *item* $i$ è funzione dell'abilità $\theta$ del solutore $j$. I tre parametri modellano le caratteristiche dell'*item*:
- $a$, detto *difficoltà* dell'item: determina il livello di abilità necessaria per avere il 50% di probabilità di successo;
- $b$, detto *differenziazione*: determina quanto influiscono le variazioni di abilità sulla probabilità di successo;
- $c$: fissa una probabilità minima di successo per qualsiasi abilità, dovuta per esempio alla possibilità di scegliere casualmente la risposta giusta ($c$ sarà 0 per un *item* a risposta completamente aperta)

In [1]:
logistic <- function(theta, a=0, b=1, c=0) { 
    c + (1-c)/(1 + exp(-b*(theta-a)))
}

In [2]:
library("ggplot2")
data <- data.frame(x=seq(-5,5,.1))
data <- transform(data, 
                  y_mean = logistic(x), 
                  y_hard = logistic(x,a=2), 
                  y_easy=logistic(x,a=-2),
                  y_hdiscrimination=logistic(x,b=2),
                  y_ldiscrimination=logistic(x,b=.5)
                 );

options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_line(aes(y=y_hard), color="red")
p <- p + annotate("text", x=2.1, y=.55, label="a == 2 ~~(plain(difficile))", parse=TRUE, color="red")
p <- p + geom_line(aes(y=y_easy), color="dark green")
p <- p + annotate("text", x=-1.9, y=.55, label="a == -2 ~~(plain(facile))", parse=TRUE, color="dark green")
p <- p + geom_line(aes(y=y_mean), color="blue")
p <- p + annotate("text", x=1.5, y=.8, label="a == 0 ~~(plain(medio))", parse=TRUE, color="blue")
p <- p + geom_line(aes(y=y_hdiscrimination), color="dark cyan")
p <- p + annotate("text", x=1, y=.9, label="a == 0 ~~ b == 2", parse=TRUE, color="dark cyan")
p <- p + geom_line(aes(y=y_ldiscrimination), color="violet")
p <- p + annotate("text", x=1.8, y=.7, label="a == 0 ~~ b == .5", parse=TRUE, color="violet")
p <- p + labs(title=expression(paste("Curve logistiche per diversi valori di ", a, " e ", b, " (",c==0,")")), 
              x="Abilità", y="Probabilità di successo")

ggsave(filename = 'img/logistic.png', plot = p, width = 6, height = 6)
rm(data, p)

![logistic](img/logistic.png)

A ogni *item* o quiz, quindi, viene associata una specifica curva logistica (la sua *item response function*) che può essere dedotta osservando un campione di risolutori cui è stato sottoposto il test. A questo scopo i parametri relativi ai quiz del Bebras italiano 2015 sono stati stimati con un approccio bayesiano, come descritto in [C. Bellettini et al. *"How challenging are Bebras tasks? an IRT analysis based on the performance of Italian students", Proceedings of the 20th annual conference on innovation and technology in computer science education ITiCSE'15* (Vilnius, Lithuania, 2015)](http://dx.doi.org/10.1145/2729094.2742603). I dettagli statistici riguardo la bontà di adattamento del modello sono disponibili [qui](https://mmonga.shinyapps.io/bebrasIT2015).

## I quesiti 2015

In [3]:
library("rstan"); rstan_options(auto_write = TRUE); options(mc.cores = parallel::detectCores())
load("fit.RData") # carica l'oggetto fit
dfit <- as.data.frame(fit)
obs <- read_rdump("bebras-all.data.R")
source("qnames.R")

cats <- c("kilo", "mega", "giga", "tera", "peta")
categories <- factor(cats, levels=cats, ordered = TRUE)
rm(cats)

Loading required package: Rcpp
rstan (Version 2.8.0, packaged: 2015-09-19 14:48:38 UTC, GitRev: 05c3d0058b6a)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())


`dfit` è il data frame che raccoglie l'esito della simulazione, con 3394 variabili (3168 *ability*, 75 *discrimination*, 75 *difficulty*, 75 *guessing* e la logprobability) e 2000 campioni. `obs` è la lista dei dati effettivamente osservati, cioè 47520 risposte di 3168 squadre a 75 quiz, con 5 categorie. `qnames` contiene i nomi completi dei quiz, come `'2015_Kilo_A1_HU02_Apparecchiare'`.

In [4]:
obs$rcats <- plyr::aaply(obs$rquiz, 1, function (e){ as.character(categories[ ((e - 1) %/% 15) + 1 ]) })

In [5]:
library("plyr")
library("dplyr")
teams <- data.frame(id = obs$rtaker, category = obs$rcats) %>% distinct()
get_par_df <- function(df, par, table) {
    df %>% 
    dplyr::select(contains(par)) %>% 
    tidyr::gather(key = "parameter", value, everything()) %>%
    dplyr::mutate("id" = as.numeric(regmatches(parameter, regexpr("[0-9]+",parameter)))) %>%
    dplyr::left_join(table, by = "id") %>%
    transform(category = factor(category, levels=levels(categories), ordered = TRUE))
}


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [6]:
team_ability <- get_par_df(dfit, "ability", teams)

Il modello è tarato in modo che la media delle abilità di tutte le squadre partecipanti sia 0.

In [7]:
print(c(summary(team_ability$value), paste("StdDev: ", sd(team_ability$value))), quote = FALSE)

                     Min.                   1st Qu.                    Median 
                   -5.616                   -0.7227                  0.002395 
                     Mean                   3rd Qu.                      Max. 
               -3.509e-17                    0.7237                     5.662 
                          
StdDev:  1.07960634648982 


In [8]:
team_ability.notes <- ddply(team_ability, .(category), summarise, note=sprintf("paste(mu == %s, ' ', sigma == %s)", 
                                                                     format(mean(value), digit = 2), 
                                                                     format(sd(value), digit= 2)))

In [9]:
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(team_ability, aes(x=value)) 
p <- p + geom_histogram(binwidth=.2) + coord_cartesian(ylim = c(0,.5))
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
p <- p + geom_text(data=team_ability.notes, aes(0, 0.4, label=note), size=3, parse = TRUE)
p <- p + labs(title="Distribuzione dell'abilità dei partecipanti", 
              x="Abilità", y="Densità")
ggsave(filename = 'img/team_ability.png', plot = p, width = 10, height = 2)

![team_ability](img/team_ability.png)

In [10]:
qorder <- plyr::laply(strsplit(qnames, "_"), function(e){e[3]})
quizzes <- data.frame(id = 1:length(qnames), quiz = qnames, 
                      category = factor(plyr::laply(strsplit(qnames, "_"), function(e){tolower(e[2])}), 
                                        levels=levels(categories), ordered = TRUE),
                      qname = plyr::laply(strsplit(qnames, "_"), function(e){e[5]}),
                      bebras_id = plyr::laply(strsplit(qnames, "_"), function(e){e[4]}),
                      ord = factor(qorder, levels=rev(sort(unique(qorder))), ordered = TRUE),
                      country = plyr::laply(strsplit(qnames, "_"), function(e){substr(e[4],1,2)}),
                      block = plyr::laply(strsplit(qnames, "_"), function(e){substr(e[3],1,1)}))
                      

In [11]:
quiz_difficulty <- get_par_df(dfit, par = "difficulty", table = quizzes)                                    

In [12]:
options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(quiz_difficulty, aes(ord, value, fill=category)) 
p <- p + geom_boxplot(outlier.shape = 1, outlier.size = .5)
p <- p + coord_flip() 
p <- p + labs(title="Distribuzione della difficoltà dei quiz", 
              x="Quiz", y="Difficoltà")
ggsave(filename = 'img/quiz_difficulty.png', plot = p, width = 10, height = 10)
rm(p)

![quiz_difficulty](img/quiz_difficulty.png)

In [13]:
quiz_discrimination <- get_par_df(dfit, par = "discrimination", table = quizzes)

In [14]:
options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(quiz_discrimination, aes(ord, value, fill=category)) 
p <- p + geom_boxplot(outlier.shape = 1, outlier.size = .5)
p <- p + coord_flip() 
p <- p + labs(title="Distribuzione della differenziazione dei quiz", 
              x="Quiz", y="Differenziazione")
ggsave(filename = 'img/quiz_discrimination.png', plot = p, width = 10, height = 10)
rm(p)

![quiz_discrimination](img/quiz_discrimination.png)

In [15]:
quiz_guessing <- get_par_df(dfit, par = "guessing", table = quizzes)

##  I singoli quesiti

###  2015_Kilo_A1_HU02_Apparecchiare

![Apparecchiare](screenshot/kilo01-collage.png)

In [16]:
q <- 'HU02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [17]:
if (!file.exists('img/d-HU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-HU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-2.2490 -1.3620 -1.2120 -1.2310 -1.0810 -0.6368 

![d-HU02](img/d-HU02.png)

#### Differenziazione

In [18]:
if (!file.exists('img/a-HU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-HU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4521  0.7814  0.8745  0.8873  0.9867  1.7020 

![a-HU02](img/a-HU02.png)

#### Guessing

In [19]:
if (!file.exists('img/g-HU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-HU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
7.590e-06 2.494e-03 5.857e-03 8.415e-03 1.179e-02 5.181e-02 

![g-HU02](img/g-HU02.png)

#### Curva Item-Response

In [20]:
if (!file.exists('img/l-HU02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-HU02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-HU02](img/l-HU02.png)

###  2015_Kilo_A2_TR03_Palloncini

![Palloncini](screenshot/kilo02-palloncini.png)

In [21]:
q <- 'TR03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [22]:
if (!file.exists('img/d-TR03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-TR03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -2.966  -1.954  -1.756  -1.782  -1.584  -1.061 

![d-TR03](img/d-TR03.png)

#### Differenziazione

In [23]:
if (!file.exists('img/a-TR03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-TR03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4189  0.7486  0.8314  0.8436  0.9345  1.4530 

![a-TR03](img/a-TR03.png)

#### Guessing

In [24]:
if (!file.exists('img/g-TR03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-TR03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
6.800e-07 3.649e-03 9.061e-03 1.263e-02 1.768e-02 1.037e-01 

![g-TR03](img/g-TR03.png)

#### Curva Item-Response

In [25]:
if (!file.exists('img/l-TR03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-TR03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-TR03](img/l-TR03.png)

###  2015_Kilo_A3_PL08_Calze

![Calze](screenshot/kilo03-calze.png)

In [26]:
q <- 'PL08'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [27]:
if (!file.exists('img/d-PL08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-PL08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2284  0.8810  1.0420  1.0530  1.2180  2.1170 

![d-PL08](img/d-PL08.png)

#### Differenziazione

In [28]:
if (!file.exists('img/a-PL08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-PL08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5682  1.0510  1.2800  1.3360  1.5480  3.6210 

![a-PL08](img/a-PL08.png)

#### Guessing

In [29]:
if (!file.exists('img/g-PL08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-PL08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.001047 0.112900 0.162200 0.157000 0.203200 0.329000 

![g-PL08](img/g-PL08.png)

#### Curva Item-Response

In [30]:
if (!file.exists('img/l-PL08.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-PL08.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-PL08](img/l-PL08.png)

###  2015_Kilo_A4_MY01_Braccialetti

![Braccialetti](screenshot/kilo04-braccialetti.png)

In [31]:
q <- 'MY01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [32]:
if (!file.exists('img/d-MY01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-MY01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-5.1240 -3.2940 -2.9000 -2.8940 -2.4880 -0.6867 

![d-MY01](img/d-MY01.png)

#### Differenziazione

In [33]:
if (!file.exists('img/a-MY01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-MY01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5407  1.1190  1.3030  1.3450  1.5250  3.1790 

![a-MY01](img/a-MY01.png)

#### Guessing

In [34]:
if (!file.exists('img/g-MY01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-MY01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000556 0.1503000 0.3175000 0.3452000 0.5164000 0.9300000 

![g-MY01](img/g-MY01.png)

#### Curva Item-Response

In [35]:
if (!file.exists('img/l-MY01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-MY01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-MY01](img/l-MY01.png)

###  2015_Kilo_A5_SK05_Vestiti

![Vestiti](screenshot/kilo05-vestiti.png)

In [36]:
q <- 'SK05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [37]:
if (!file.exists('img/d-SK05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-4.6820 -1.8990 -1.0170 -0.9968 -0.1550  3.5530 

![d-SK05](img/d-SK05.png)

#### Differenziazione

In [38]:
if (!file.exists('img/a-SK05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.3410  0.6416  0.7580  0.7917  0.9038  2.4740 

![a-SK05](img/a-SK05.png)

#### Guessing

In [39]:
if (!file.exists('img/g-SK05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0006019 0.3984000 0.5942000 0.5361000 0.7061000 0.8742000 

![g-SK05](img/g-SK05.png)

#### Curva Item-Response

In [40]:
if (!file.exists('img/l-SK05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK05](img/l-SK05.png)

###  2015_Kilo_B1_LT05_Morra

![Morra](screenshot/kilo06-morra.png)

In [41]:
q <- 'LT05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [42]:
if (!file.exists('img/d-LT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-LT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-1.640000 -0.981900 -0.362700 -0.481300  0.002391  0.571300 

![d-LT05](img/d-LT05.png)

#### Differenziazione

In [43]:
if (!file.exists('img/a-LT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-LT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.7661  1.2350  1.3890  1.4390  1.5890  3.2630 

![a-LT05](img/a-LT05.png)

#### Guessing

In [44]:
if (!file.exists('img/g-LT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-LT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000484 0.0265400 0.0618200 0.0757100 0.1090000 0.3917000 

![g-LT05](img/g-LT05.png)

#### Curva Item-Response

In [45]:
if (!file.exists('img/l-LT05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-LT05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-LT05](img/l-LT05.png)

###  2015_Kilo_B2_JP04_Gru

![Gru](screenshot/kilo07-gru.png)

In [46]:
q <- 'JP04'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [47]:
if (!file.exists('img/d-JP04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-JP04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.9001  1.4020  1.7230  1.7730  2.0980  3.6400 

![d-JP04](img/d-JP04.png)

#### Differenziazione

In [48]:
if (!file.exists('img/a-JP04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-JP04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4927  0.8696  0.9883  1.0170  1.1380  2.0580 

![a-JP04](img/a-JP04.png)

#### Guessing

In [49]:
if (!file.exists('img/g-JP04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-JP04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
7.500e-07 2.692e-04 6.772e-04 9.962e-04 1.384e-03 1.017e-02 

![g-JP04](img/g-JP04.png)

#### Curva Item-Response

In [50]:
if (!file.exists('img/l-JP04.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-JP04.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-JP04](img/l-JP04.png)

###  2015_Kilo_B3_TR08_Occhiali

![Occhiali](screenshot/kilo08-occhiali.png)

In [51]:
q <- 'TR08'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [52]:
if (!file.exists('img/d-TR08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-TR08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4275  1.1100  1.7270  2.0100  2.8190  5.5150 

![d-TR08](img/d-TR08.png)

#### Differenziazione

In [53]:
if (!file.exists('img/a-TR08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-TR08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.3814  0.9510  1.1370  1.1840  1.3620  3.2990 

![a-TR08](img/a-TR08.png)

#### Guessing

In [54]:
if (!file.exists('img/g-TR08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-TR08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000859 0.119700 0.168000 0.160700 0.208900 0.305500 

![g-TR08](img/g-TR08.png)

#### Curva Item-Response

In [55]:
if (!file.exists('img/l-TR08.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-TR08.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-TR08](img/l-TR08.png)

###  2015_Kilo_B4_AT03_Irrigazione

![Irrigazione](screenshot/kilo09-irrigazione.png)

In [56]:
q <- 'AT03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [57]:
if (!file.exists('img/d-AT03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-AT03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-2.1860 -1.3370 -0.5249 -0.5906  0.1673  0.7063 

![d-AT03](img/d-AT03.png)

#### Differenziazione

In [58]:
if (!file.exists('img/a-AT03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-AT03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4376  0.8180  0.9057  0.9118  1.0000  1.5300 

![a-AT03](img/a-AT03.png)

#### Guessing

In [59]:
if (!file.exists('img/g-AT03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-AT03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
3.800e-07 1.534e-04 3.468e-04 4.908e-04 6.880e-04 4.897e-03 

![g-AT03](img/g-AT03.png)

#### Curva Item-Response

In [60]:
if (!file.exists('img/l-AT03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-AT03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-AT03](img/l-AT03.png)

###  2015_Kilo_B5_JP01_Attrezzi

![Attrezzi](screenshot/kilo10-attrezzi.png)

In [61]:
q <- 'JP01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [62]:
if (!file.exists('img/d-JP01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-JP01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.9460 -0.8210 -0.4622 -0.4551 -0.1440  1.6810 

![d-JP01](img/d-JP01.png)

#### Differenziazione

In [63]:
if (!file.exists('img/a-JP01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-JP01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4368  0.8322  0.9599  0.9841  1.1080  3.0480 

![a-JP01](img/a-JP01.png)

#### Guessing

In [64]:
if (!file.exists('img/g-JP01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-JP01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000097 0.087650 0.170900 0.182800 0.264700 0.555200 

![g-JP01](img/g-JP01.png)

#### Curva Item-Response

In [65]:
if (!file.exists('img/l-JP01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-JP01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-JP01](img/l-JP01.png)

###  2015_Kilo_C1_CZ03_Trasporto

![Trasporto](screenshot/kilo11-trasporto.png)

In [66]:
q <- 'CZ03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [67]:
if (!file.exists('img/d-CZ03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CZ03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.253   2.061   2.362   2.812   3.630   7.177 

![d-CZ03](img/d-CZ03.png)

#### Differenziazione

In [68]:
if (!file.exists('img/a-CZ03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CZ03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4206  0.8827  1.1180  1.1800  1.4390  2.8980 

![a-CZ03](img/a-CZ03.png)

#### Guessing

In [69]:
if (!file.exists('img/g-CZ03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CZ03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000017 0.0011950 0.0027930 0.0038340 0.0052530 0.0363800 

![g-CZ03](img/g-CZ03.png)

#### Curva Item-Response

In [70]:
if (!file.exists('img/l-CZ03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CZ03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CZ03](img/l-CZ03.png)

###  2015_Kilo_C2_SE01_Stelle

![Stelle](screenshot/kilo12-disegnare.png)

In [71]:
q <- 'SE01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [72]:
if (!file.exists('img/d-SE01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SE01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-2.57900 -1.11700 -0.03978 -0.10280  0.87250  2.33500 

![d-SE01](img/d-SE01.png)

#### Differenziazione

In [73]:
if (!file.exists('img/a-SE01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SE01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.3553  0.7457  0.8738  0.8785  1.0030  1.5210 

![a-SE01](img/a-SE01.png)

#### Guessing

In [74]:
if (!file.exists('img/g-SE01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SE01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
2.380e-07 2.841e-04 6.958e-04 9.984e-04 1.370e-03 7.278e-03 

![g-SE01](img/g-SE01.png)

#### Curva Item-Response

In [75]:
if (!file.exists('img/l-SE01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SE01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SE01](img/l-SE01.png)

###  2015_Kilo_C3_CZ07_Drone

![Drone](screenshot/kilo13-drone.png)

In [76]:
q <- 'CZ07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [77]:
if (!file.exists('img/d-CZ07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CZ07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -1.320   0.715   1.149   1.470   2.473   5.743 

![d-CZ07](img/d-CZ07.png)

#### Differenziazione

In [78]:
if (!file.exists('img/a-CZ07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CZ07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.3245  0.7209  0.8593  0.8983  1.0300  2.9090 

![a-CZ07](img/a-CZ07.png)

#### Guessing

In [79]:
if (!file.exists('img/g-CZ07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CZ07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000579 0.0682600 0.1282000 0.1667000 0.2311000 0.5843000 

![g-CZ07](img/g-CZ07.png)

#### Curva Item-Response

In [80]:
if (!file.exists('img/l-CZ07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CZ07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CZ07](img/l-CZ07.png)

###  2015_Kilo_C4_CZ01_Noccioline

![Noccioline](screenshot/kilo14-noci.png)

In [81]:
q <- 'CZ01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [82]:
if (!file.exists('img/d-CZ01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CZ01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-3.0230 -1.0890  0.2513  0.1648  1.3690  2.7320 

![d-CZ01](img/d-CZ01.png)

#### Differenziazione

In [83]:
if (!file.exists('img/a-CZ01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CZ01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2706  0.6971  0.8623  0.8613  1.0090  2.7250 

![a-CZ01](img/a-CZ01.png)

#### Guessing

In [84]:
if (!file.exists('img/g-CZ01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CZ01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000079 0.0134100 0.0317400 0.0431300 0.0626000 0.3718000 

![g-CZ01](img/g-CZ01.png)

#### Curva Item-Response

In [85]:
if (!file.exists('img/l-CZ01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CZ01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CZ01](img/l-CZ01.png)

###  2015_Kilo_C5_AT07_Elenco

![Elenco](screenshot/kilo15-elenco.png)

In [86]:
q <- 'AT07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [87]:
if (!file.exists('img/d-AT07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-AT07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.8843  2.2040  2.6690  2.6490  3.0680  5.6560 

![d-AT07](img/d-AT07.png)

#### Differenziazione

In [88]:
if (!file.exists('img/a-AT07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-AT07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2695  0.6380  0.8115  0.9320  1.1260  4.2390 

![a-AT07](img/a-AT07.png)

#### Guessing

In [89]:
if (!file.exists('img/g-AT07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-AT07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
8.714e-05 3.152e-02 5.509e-02 6.133e-02 8.152e-02 2.480e-01 

![g-AT07](img/g-AT07.png)

#### Curva Item-Response

In [90]:
if (!file.exists('img/l-AT07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-AT07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-AT07](img/l-AT07.png)

###  2015_Mega_D1_SK02_Tessuto

![Tessuto](screenshot/mega11-tessuto.png)

In [91]:
q <- 'SK02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [92]:
if (!file.exists('img/d-SK02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-0.2290  0.2195  0.3710  0.6017  1.1360  1.7260 

![d-SK02](img/d-SK02.png)

#### Differenziazione

In [93]:
if (!file.exists('img/a-SK02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.3243  0.8165  0.9398  1.0270  1.2490  2.2070 

![a-SK02](img/a-SK02.png)

#### Guessing

In [94]:
if (!file.exists('img/g-SK02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
2.700e-12 1.736e-08 4.112e-08 5.921e-08 8.252e-08 7.576e-07 

![g-SK02](img/g-SK02.png)

#### Curva Item-Response

In [95]:
if (!file.exists('img/l-SK02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK02](img/l-SK02.png)

###  2015_Mega_D2_IT02_Meteo

![Meteo](screenshot/mega12-meteo.png)

In [96]:
q <- 'IT02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [97]:
if (!file.exists('img/d-IT02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-IT02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.05231 1.01500 1.49200 1.60000 2.17600 4.71300 

![d-IT02](img/d-IT02.png)

#### Differenziazione

In [98]:
if (!file.exists('img/a-IT02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-IT02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.3266  0.7572  0.9135  0.9522  1.1010  2.5250 

![a-IT02](img/a-IT02.png)

#### Guessing

In [99]:
if (!file.exists('img/g-IT02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-IT02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00090 0.09864 0.15270 0.14960 0.19820 0.35970 

![g-IT02](img/g-IT02.png)

#### Curva Item-Response

In [100]:
if (!file.exists('img/l-IT02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-IT02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-IT02](img/l-IT02.png)

###  2015_Mega_D3_BE04_Scoprirai

![Scoprirai](screenshot/mega13-scoprirai.png)

In [101]:
q <- 'BE04'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [102]:
if (!file.exists('img/d-BE04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-BE04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-5.0800 -2.6980 -1.7710 -1.8490 -0.9060  0.3961 

![d-BE04](img/d-BE04.png)

#### Differenziazione

In [103]:
if (!file.exists('img/a-BE04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-BE04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.520   0.972   1.144   1.213   1.391   3.491 

![a-BE04](img/a-BE04.png)

#### Guessing

In [104]:
if (!file.exists('img/g-BE04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-BE04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000191 0.0803800 0.1719000 0.2086000 0.2996000 0.8296000 

![g-BE04](img/g-BE04.png)

#### Curva Item-Response

In [105]:
if (!file.exists('img/l-BE04.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-BE04.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-BE04](img/l-BE04.png)

###  2015_Mega_D4_RU07_Alchimista

![Alchimista](screenshot/mega14-alchimista.png)

In [106]:
q <- 'RU07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [107]:
if (!file.exists('img/d-RU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-RU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-0.46210 -0.09697  1.24900  1.21700  2.30000  3.70400 

![d-RU07](img/d-RU07.png)

#### Differenziazione

In [108]:
if (!file.exists('img/a-RU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-RU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5227  0.8705  0.9633  0.9739  1.0640  1.6460 

![a-RU07](img/a-RU07.png)

#### Guessing

In [109]:
if (!file.exists('img/g-RU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-RU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
8.800e-08 2.825e-04 6.861e-04 1.003e-03 1.375e-03 8.201e-03 

![g-RU07](img/g-RU07.png)

#### Curva Item-Response

In [110]:
if (!file.exists('img/l-RU07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-RU07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-RU07](img/l-RU07.png)

###  2015_Mega_D5_HU05_Acrobati

![Acrobati](screenshot/mega15-acrobati.png)

In [111]:
q <- 'HU05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [112]:
if (!file.exists('img/d-HU05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-HU05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.4760 -0.5385  0.1331  0.2426  1.0740  2.3340 

![d-HU05](img/d-HU05.png)

#### Differenziazione

In [113]:
if (!file.exists('img/a-HU05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-HU05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4233  0.8540  0.9804  0.9897  1.1070  1.9050 

![a-HU05](img/a-HU05.png)

#### Guessing

In [114]:
if (!file.exists('img/g-HU05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-HU05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0001163 0.0484600 0.0995500 0.1109000 0.1597000 0.4335000 

![g-HU05](img/g-HU05.png)

#### Curva Item-Response

In [115]:
if (!file.exists('img/l-HU05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-HU05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-HU05](img/l-HU05.png)

###  2015_Giga_E1_SK04_Luci

![Luci](screenshot/giga11-luci.png)

In [116]:
q <- 'SK04'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [117]:
if (!file.exists('img/d-SK04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-3.0900 -1.5150 -0.5072 -0.4768  0.7164  1.5140 

![d-SK04](img/d-SK04.png)

#### Differenziazione

In [118]:
if (!file.exists('img/a-SK04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2393  0.5577  0.7063  0.7951  1.0180  1.8810 

![a-SK04](img/a-SK04.png)

#### Guessing

In [119]:
if (!file.exists('img/g-SK04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
3.510e-09 8.540e-06 2.088e-05 3.068e-05 4.277e-05 2.795e-04 

![g-SK04](img/g-SK04.png)

#### Curva Item-Response

In [120]:
if (!file.exists('img/l-SK04.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK04.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK04](img/l-SK04.png)

###  2015_Giga_E2_CH09_Labirinti

![Labirinti](screenshot/giga12-labirinti.png)

In [121]:
q <- 'CH09'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [122]:
if (!file.exists('img/d-CH09.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CH09.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  2.748   4.101   4.565   4.655   5.100  10.020 

![d-CH09](img/d-CH09.png)

#### Differenziazione

In [123]:
if (!file.exists('img/a-CH09.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CH09.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4738  0.9852  1.1910  1.2550  1.4530  3.7480 

![a-CH09](img/a-CH09.png)

#### Guessing

In [124]:
if (!file.exists('img/g-CH09.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CH09.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
6.270e-06 9.778e-03 1.580e-02 1.911e-02 2.674e-02 7.190e-02 

![g-CH09](img/g-CH09.png)

#### Curva Item-Response

In [125]:
if (!file.exists('img/l-CH09.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CH09.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CH09](img/l-CH09.png)

###  2015_Giga_E3_HU07_Pianoforte

![Pianoforte](screenshot/giga13-pianoforte.png)

In [126]:
q <- 'HU07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [127]:
if (!file.exists('img/d-HU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-HU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.8660 -0.6407  0.4960  0.5624  1.5780  5.9860 

![d-HU07](img/d-HU07.png)

#### Differenziazione

In [128]:
if (!file.exists('img/a-HU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-HU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2965  0.8319  0.9706  1.0040  1.1320  3.7690 

![a-HU07](img/a-HU07.png)

#### Guessing

In [129]:
if (!file.exists('img/g-HU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-HU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000354 0.1563000 0.2692000 0.2583000 0.3624000 0.5332000 

![g-HU07](img/g-HU07.png)

#### Curva Item-Response

In [130]:
if (!file.exists('img/l-HU07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-HU07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-HU07](img/l-HU07.png)

###  2015_Giga_E4_CA01_Popolarita

![Popolarita](screenshot/giga14-popolarita.png)

In [131]:
q <- 'CA01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [132]:
if (!file.exists('img/d-CA01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CA01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-1.50800 -0.07661  0.60320  0.63250  1.42300  3.70700 

![d-CA01](img/d-CA01.png)

#### Differenziazione

In [133]:
if (!file.exists('img/a-CA01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CA01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2804  0.6069  0.7040  0.7336  0.8299  1.7770 

![a-CA01](img/a-CA01.png)

#### Guessing

In [134]:
if (!file.exists('img/g-CA01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CA01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000002 0.0014160 0.0914100 0.1194000 0.2160000 0.5220000 

![g-CA01](img/g-CA01.png)

#### Curva Item-Response

In [135]:
if (!file.exists('img/l-CA01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CA01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CA01](img/l-CA01.png)

###  2015_Giga_E5_TW03_LuogoDelitto

![LuogoDelitto](screenshot/giga15-delitto.png)

In [136]:
q <- 'TW03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [137]:
if (!file.exists('img/d-TW03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-TW03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-2.1400 -0.6627  0.1412  0.0726  0.8392  1.7720 

![d-TW03](img/d-TW03.png)

#### Differenziazione

In [138]:
if (!file.exists('img/a-TW03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-TW03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5974  1.0820  1.2210  1.2420  1.3760  3.0760 

![a-TW03](img/a-TW03.png)

#### Guessing

In [139]:
if (!file.exists('img/g-TW03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-TW03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000011 0.0012930 0.0409600 0.0938000 0.1389000 0.5983000 

![g-TW03](img/g-TW03.png)

#### Curva Item-Response

In [140]:
if (!file.exists('img/l-TW03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-TW03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-TW03](img/l-TW03.png)

###  2015_Tera_F1_SK01_Robot

![Robot](screenshot/tera11-robot.png)

In [141]:
q <- 'SK01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [142]:
if (!file.exists('img/d-SK01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.569   2.277   2.720   2.750   3.139   5.122 

![d-SK01](img/d-SK01.png)

#### Differenziazione

In [143]:
if (!file.exists('img/a-SK01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5047  0.8603  1.0980  1.2440  1.5890  3.3540 

![a-SK01](img/a-SK01.png)

#### Guessing

In [144]:
if (!file.exists('img/g-SK01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
1.400e-08 2.748e-04 6.376e-04 9.102e-04 1.256e-03 8.257e-03 

![g-SK01](img/g-SK01.png)

#### Curva Item-Response

In [145]:
if (!file.exists('img/l-SK01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK01](img/l-SK01.png)

###  2015_Tera_F2_CA02_Fuochi

![Fuochi](screenshot/tera12-fuochi.png)

In [146]:
q <- 'CA02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [147]:
if (!file.exists('img/d-CA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.8990 -1.0760 -0.2238 -0.3330  0.4257  0.7899 

![d-CA02](img/d-CA02.png)

#### Differenziazione

In [148]:
if (!file.exists('img/a-CA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4284  0.7883  0.9261  0.9300  1.0600  1.5740 

![a-CA02](img/a-CA02.png)

#### Guessing

In [149]:
if (!file.exists('img/g-CA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
3.010e-07 2.898e-04 6.922e-04 1.003e-03 1.396e-03 9.126e-03 

![g-CA02](img/g-CA02.png)

#### Curva Item-Response

In [150]:
if (!file.exists('img/l-CA02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CA02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CA02](img/l-CA02.png)

###  2015_Tera_F3_RU02_Ricetta

![Ricetta](screenshot/tera13-ricetta.png)

In [151]:
q <- 'RU02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [152]:
if (!file.exists('img/d-RU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-RU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-0.4501 -0.1561  0.5263  0.5564  1.2690  1.8370 

![d-RU02](img/d-RU02.png)

#### Differenziazione

In [153]:
if (!file.exists('img/a-RU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-RU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.703   1.106   1.206   1.214   1.318   1.847 

![a-RU02](img/a-RU02.png)

#### Guessing

In [154]:
if (!file.exists('img/g-RU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-RU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
5.650e-07 2.878e-04 6.958e-04 9.830e-04 1.367e-03 8.861e-03 

![g-RU02](img/g-RU02.png)

#### Curva Item-Response

In [155]:
if (!file.exists('img/l-RU02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-RU02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-RU02](img/l-RU02.png)

###  2015_Tera_F4_AT01_Famiglia

![Famiglia](screenshot/tera14-famiglia.png)

In [156]:
q <- 'AT01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [157]:
if (!file.exists('img/d-AT01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-AT01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4512  0.7680  1.5790  1.8200  2.8420  4.2940 

![d-AT01](img/d-AT01.png)

#### Differenziazione

In [158]:
if (!file.exists('img/a-AT01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-AT01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.6245  1.0540  1.1690  1.1770  1.2930  1.8470 

![a-AT01](img/a-AT01.png)

#### Guessing

In [159]:
if (!file.exists('img/g-AT01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-AT01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
4.210e-06 1.766e-03 4.073e-03 5.770e-03 7.945e-03 4.221e-02 

![g-AT01](img/g-AT01.png)

#### Curva Item-Response

In [160]:
if (!file.exists('img/l-AT01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-AT01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-AT01](img/l-AT01.png)

###  2015_Tera_F5_UA02_Bibite

![Bibite](screenshot/tera15-bibite.png)

In [161]:
q <- 'UA02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [162]:
if (!file.exists('img/d-UA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-UA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.6784  1.0460  1.9670  2.3230  3.5580  5.8330 

![d-UA02](img/d-UA02.png)

#### Differenziazione

In [163]:
if (!file.exists('img/a-UA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-UA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4395  0.7309  0.9325  0.9509  1.1620  1.6370 

![a-UA02](img/a-UA02.png)

#### Guessing

In [164]:
if (!file.exists('img/g-UA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-UA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
1.145e-08 4.484e-06 1.047e-05 1.521e-05 2.119e-05 1.025e-04 

![g-UA02](img/g-UA02.png)

#### Curva Item-Response

In [165]:
if (!file.exists('img/l-UA02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-UA02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-UA02](img/l-UA02.png)

###  2015_Peta_G1_CH01_Calcolatrice

![Calcolatrice](screenshot/peta11-calcolatrice.png)

In [166]:
q <- 'CH01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [167]:
if (!file.exists('img/d-CH01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CH01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  2.340   2.968   3.270   3.323   3.604   5.764 

![d-CH01](img/d-CH01.png)

#### Differenziazione

In [168]:
if (!file.exists('img/a-CH01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CH01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4351  0.7378  0.8328  0.8384  0.9294  1.4150 

![a-CH01](img/a-CH01.png)

#### Guessing

In [169]:
if (!file.exists('img/g-CH01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CH01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
3.710e-07 2.614e-04 6.525e-04 9.904e-04 1.412e-03 7.196e-03 

![g-CH01](img/g-CH01.png)

#### Curva Item-Response

In [170]:
if (!file.exists('img/l-CH01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CH01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CH01](img/l-CH01.png)

###  2015_Peta_G2_DE05_Ciondoli

![Ciondoli](screenshot/peta12-ciondoli.png)

In [171]:
q <- 'DE05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [172]:
if (!file.exists('img/d-DE05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-DE05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.121   1.518   1.618   1.623   1.719   2.451 

![d-DE05](img/d-DE05.png)

#### Differenziazione

In [173]:
if (!file.exists('img/a-DE05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-DE05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.110   1.659   1.826   1.855   2.024   2.964 

![a-DE05](img/a-DE05.png)

#### Guessing

In [174]:
if (!file.exists('img/g-DE05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-DE05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
1.039e-06 2.509e-04 6.267e-04 9.057e-04 1.273e-03 6.737e-03 

![g-DE05](img/g-DE05.png)

#### Curva Item-Response

In [175]:
if (!file.exists('img/l-DE05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-DE05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-DE05](img/l-DE05.png)

###  2015_Peta_G3_IT05_Sassolini

![Sassolini](screenshot/peta13-sassolini.png)

In [176]:
q <- 'IT05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [177]:
if (!file.exists('img/d-IT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-IT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.6506  0.9262  1.0010  1.0090  1.0870  1.4470 

![d-IT05](img/d-IT05.png)

#### Differenziazione

In [178]:
if (!file.exists('img/a-IT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-IT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.8461  1.2690  1.3840  1.3930  1.5020  2.2380 

![a-IT05](img/a-IT05.png)

#### Guessing

In [179]:
if (!file.exists('img/g-IT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-IT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
8.620e-07 2.708e-04 6.790e-04 1.011e-03 1.412e-03 8.334e-03 

![g-IT05](img/g-IT05.png)

#### Curva Item-Response

In [180]:
if (!file.exists('img/l-IT05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-IT05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-IT05](img/l-IT05.png)

###  2015_Peta_G4_JP05_Agenzia

![Agenzia](screenshot/peta14-agenzia.png)

In [181]:
q <- 'JP05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [182]:
if (!file.exists('img/d-JP05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-JP05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.9773  1.3300  1.4390  1.4470  1.5510  2.1360 

![d-JP05](img/d-JP05.png)

#### Differenziazione

In [183]:
if (!file.exists('img/a-JP05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-JP05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.6681  1.1360  1.2960  1.3460  1.5120  2.9980 

![a-JP05](img/a-JP05.png)

#### Guessing

In [184]:
if (!file.exists('img/g-JP05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-JP05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
8.238e-05 3.792e-02 6.229e-02 6.329e-02 8.757e-02 1.561e-01 

![g-JP05](img/g-JP05.png)

#### Curva Item-Response

In [185]:
if (!file.exists('img/l-JP05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-JP05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-JP05](img/l-JP05.png)

###  2015_Peta_G5_CA06_Carte

![Carte](screenshot/peta15-carte.png)

In [186]:
q <- 'CA06'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [187]:
if (!file.exists('img/d-CA06.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CA06.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  2.316   3.007   3.281   3.321   3.588   5.462 

![d-CA06](img/d-CA06.png)

#### Differenziazione

In [188]:
if (!file.exists('img/a-CA06.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CA06.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.6244  1.0120  1.1370  1.1520  1.2720  1.9580 

![a-CA06](img/a-CA06.png)

#### Guessing

In [189]:
if (!file.exists('img/g-CA06.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CA06.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
1.373e-06 2.634e-04 6.356e-04 9.165e-04 1.301e-03 6.417e-03 

![g-CA06](img/g-CA06.png)

#### Curva Item-Response

In [190]:
if (!file.exists('img/l-CA06.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CA06.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CA06](img/l-CA06.png)